# Personalized Job-Based Cold Email Generator

#### This project leverages the power of Large Language Models (LLMs), specifically LLaMA 3.1 via Groq, to automatically generate personalized cold emails for job applications. Instead of relying on predefined CSV portfolios or keyword matching, this system:

🔹 Extracts detailed information from a job posting URL  
🔹 Parses and analyzes your uploaded resume (PDF or DOCX)  
🔹 Crafts a compelling, role-specific cold email tailored to your skills and experience  

It combines natural language understanding with your professional profile to help you stand out to recruiters and hiring managers — all with just one click.


In [1]:
# Importing Libraries

import re
from PyPDF2 import PdfReader
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Function for Text Cleaning

def clean_text(text):
    text = re.sub(r'\n\s*\n+', '\n\n', text)
    return '\n'.join(line.strip() for line in text.strip().splitlines())

# Function for Loading Resume

def load_resume(file_path):
    if file_path.endswith(".pdf"):
        reader = PdfReader(file_path)
        page_content = ""
        for page in reader.pages:
            page_content += page.extract_text()
        return page_content
    elif file_path.endswith(".docx"):
        doc = Document(file_path)
        return "\n".join([paragraph.text for paragraph in doc.paragraphs])
    else:
        raise ValueError("Unsupported file type")


In [ ]:
# Class for Cold Email Generation

class ColdEmailGenerator:
    def __init__(self):
        self.llm = ChatGroq(
                            temperature = 0.2, # Adjust temperature for creativity
                            groq_api_key = 'gskxxxxxxxxxxxxxxxxxxxxxxxx', # Add your Groq API key here
                            model_name = "llama-3.3-70b-versatile" # Change to your desired model
                            )

    def extract_jobs(self, page_data):  # Prompt for extracting job postings
        prompt = PromptTemplate.from_template(
            """
            ### SCRAPED TEXT FROM WEBSITE:
            {page_data}
            ### INSTRUCTION:
            Extract job postings and return them in JSON with: `role`, `experience`, `skills`, and `description`.
            ### OUTPUT:
            """
        )
        
        chain = prompt | self.llm # Creating a chain with the Prompt and LLM
        res = chain.invoke({"page_data": page_data})
        try:
            return JsonOutputParser().parse(res.content)
        except OutputParserException:
            raise OutputParserException("Couldn't parse job description.")
    
    def generate_email(self, job_info, resume_text): # Prompt for generating cold email
        prompt = PromptTemplate.from_template(
            """
            ### JOB DESCRIPTION:
            {job_description}

            ### CANDIDATE RESUME:
            {resume_text}

            ### INSTRUCTION:
            You are a job applicant writing a cold email to express interest in the above job. 
            Highlight why you're a great fit based on your resume and background. 
            Make it concise, professional, and specific. No generic fluff.

            ### COLD EMAIL:
            """
        )
        chain = prompt | self.llm
        res = chain.invoke({"job_description": str(job_info), "resume_text": resume_text})
        return res.content

In [4]:
# Example Usage

job_url = "https://careers.nike.com/designer-ii-apparel-graphics-sport/job/R-49261" 
loader = WebBaseLoader([job_url])
page_text = loader.load().pop().page_content
cleaned_job_text = clean_text(page_text)

email_gen = ColdEmailGenerator()
job_info = email_gen.extract_jobs(cleaned_job_text)
job_info = job_info if isinstance(job_info, list) else [job_info]

job_info[0]

{'job_posting': {'role': 'Designer II, Apparel Graphics, Sport',
  'experience': '1-4+ years of relevant Product Graphic Design experience',
  'skills': ['Bachelor’s degree in graphic design, fashion design, product design, industrial design, or equivalent combination of relevant education, experience and training',
   'Ability to create seasonal Graphic and Print designs and concepts, and explore new Graphic Design innovations and applications',
   'Understanding of the Apparel Product Creation process, Footwear Product Creation experience a plus',
   'Experience working and communicating directly with manufacturing partners to ensure the accuracy of Graphics application and execution',
   'Ability to create and edit files using Adobe programs including Photoshop and Illustrator',
   'Experience building inspiration/concept presentations, and mood boards for seasonal concepts and direction',
   'Ability to present new ideas or seasonal concepts to small and large groups'],
  'descript

In [5]:
# Loading Resume

resume_raw = load_resume("/home/bhxveshhh/ML/LangChain/Cold Email Generator/testresume.pdf")
print(resume_raw)

In [6]:
# Generating Cold Email

cold_email = email_gen.generate_email(job_info[0], resume_raw)
print(cold_email)

Subject: Application for Designer II, Apparel Graphics, Sport Role

Dear Hiring Manager,

I am excited to express my interest in the Designer II, Apparel Graphics, Sport position at Jordan. With 3+ years of experience in Product Graphic Design and a Bachelor's degree in Graphic Design, I am confident that my skills and background make me a strong fit for this role.

As a seasoned graphic designer, I have a proven track record of creating innovative and seasonal graphic designs, having worked on multiple projects that involved exploring new design applications and presenting concepts to both small and large groups. My experience working with manufacturing partners to ensure accurate graphic execution has also honed my attention to detail and communication skills.

Proficient in Adobe Creative Suite, including Photoshop and Illustrator, I have a solid foundation in designing and editing files for apparel and footwear products. My understanding of the product creation process, combined wi